
##**ChipNet**

This notebook demonstrates the implementation of the paper - **ChipNet: Budget-Aware Pruning with Heaviside Continuous Approximations**

Steps to train a baseline model and then compress it for a given budget are as follows: 
*   Load the YAML file.
*   Load dataset and create dataloaders.
*   Create ChipNet object and pass the parameters in the form of a dictionary.
*   Pass the dataloaders into the compress_model method to obtain the compressed model.

Since this is a demo notebook the number of epochs have been set to 1, 1 and 2 respectively for pretraining, pruning and finetuning respectively.



In [1]:
#!git clone https://github.com/hachons/trailmet.git

In [5]:
cd /workspace/trailmet/trailmet

/workspace/trailmet/trailmet


In [6]:
!git pull

Updating 0d4d37c..e1e6e48
error: Your local changes to the following files would be overwritten by merge:
	trailmet/algorithms/algorithms.py
Please commit your changes or stash them before you merge.
error: The following untracked working tree files would be overwritten by merge:
	experiments/pruning/chip/chip.yaml
	trailmet/algorithms/prune/chip.py
Please move or remove them before you merge.
Aborting


In [7]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
import yaml

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
torch.cuda.device_count()

2

In [10]:
root = "/workspace/trailmet/trailmet/experiments/pruning/chipnet"

###Loading the yaml file for the configurations.

In [11]:
import os
with open(os.path.join(root, "chipnet.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
print(data_loaded)

{'CHIPNET_ARGS': {'BUDGET_TYPE': 'channel_ratio', 'TARGET_BUDGET': 0.5, 'BUDGET_LOSS_WEIGHTAGE': 30, 'CRISPNESS_LOSS_WEIGHTAGE': 10, 'BETA_INCREMENT': 5, 'GAMMA_INCREMENT': 2, 'STEEPNESS': 100}, 'PRETRAIN': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}, 'HARD_PRUNE': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.001, 'WEIGHT_DECAY': 0.001}, 'FINETUNE': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}}


###Loading the model.

In [12]:
model = ModelsFactory.create_model('resnet50', 100, False, insize=32)

###Loading the dataset.

In [13]:
from trailmet.datasets.classification import DatasetFactory
data_root = "/content/data_dir"

In [14]:
mkdir /content/data_dir

mkdir: cannot create directory ‘/content/data_dir’: File exists


In [15]:
train_transform = transforms.Compose(
[transforms.Resize(256),transforms.ToTensor()])


val_transform = transforms.Compose(
[transforms.Resize(256),transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.Resize(256),transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


chest_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


###Creating the dataloaders.

In [16]:
train_loader = torch.utils.data.DataLoader(
        chest_dataset['train'], batch_size=64, 
        sampler=chest_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        chest_dataset['val'], batch_size=64, 
        sampler=chest_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        chest_dataset['test'], batch_size=64, 
        sampler=chest_dataset['test_sampler'],
        num_workers=0
    )

###Creating the method's object followed by compression

In [17]:
from trailmet.algorithms.prune.chipnet import ChipNet

In [18]:
a = ChipNet(model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized

In [ ]:
a.compress_model()

In [19]:
torch.cuda.is_available()

False